# Tweepy
> Author: Andrew Eng | Date: 2020-10-05

## Objectives
This notebook serves as a scratch pad for learning how to use [Tweepy](http://docs.tweepy.org/en/latest/index.html).  

In [3]:
# Import tweepy as tw
import tweepy as tw

from datetime import datetime as dt
from elasticsearch import Elasticsearch

## Authentication

I setup my api credentials in a text file called twitter.keys in /home/andrew.  This is so my api keys won't accidentally upload when I git push the repository to the public.

The format of my twitter.keys is:

api_key = <key>
    
api_secret_key = <key>

access_token = <key>

access_token_secret = <key>

In [4]:
# Import keys from a saved file instead of inputting it directly into the script.  
# Strip whitespaces and split on = as I only want the key values

key_location = "/tmp/twitter.keys"
apikeys = []
with open(key_location) as keys:
    for i in keys:
        apikeys.append(i.split("=")[1].strip(" ").strip("\n"))
keys.close()

# Initialize dictionary
twitter_cred = dict()

# Enter API keys
twitter_cred["CONSUMER_KEY"] = apikeys[0]
twitter_cred["CONSUMER_SECRET"] = apikeys[1]

# Access Tokens
twitter_cred["ACCESS_KEY"] = apikeys[2]
twitter_cred["ACCESS_SECRET"] = apikeys[3]

# Set authentication object

auth = tw.OAuthHandler(twitter_cred["CONSUMER_KEY"], twitter_cred["CONSUMER_SECRET"])
auth.set_access_token(twitter_cred["ACCESS_KEY"], twitter_cred["ACCESS_SECRET"])

# Create api object with authentication

api = tw.API(auth, wait_on_rate_limit=True)

es = Elasticsearch('127.0.0.1', port=9200)

## Hello World

To test out if my api keys are working, let's grab some tweets from my timeline using home_timeline() method

In [5]:
public_tweets = api.home_timeline()
count = 1
for tweet in public_tweets:
    print(f"TWEET #{count} - {tweet.text} \n")
    count +=1

TWEET #1 - 💫 Newly listed cryptos: EasyFi has been added to CoinMarketCap! 

@EasyfiNetwork is a layer 2 lending protocol for… https://t.co/pNMG4h5REb 

TWEET #2 - [Course Video] Attacking and Defending Active Directory: Domain Privilege Escalation Kerberoast… https://t.co/K8N7JlQY3Z 

TWEET #3 - RT @BitTorrent: 📢Good news for #BTFS users:
 
💡#BTFS Miners will receive more rewards. From Oct 6 on, 30,000,000 $BTT will be airdropped to… 

TWEET #4 - AttackDefense Labs | Remote Code Execution: SQL Buddy https://t.co/OS7j1n0ErZ https://t.co/HX0r9bB6XC Learn More About #AttackDefense Today! 

TWEET #5 - Feeling lucky?

Share your #Binance Launchpad ticket #s for the $ALPHA token sale for a chance to win 1 of 15 prize… https://t.co/kwD5QE9Osb 

TWEET #6 - The @AlphaFinanceLab $ALPHA Token Sale &amp; Ticket Claim Will Begin Shortly

➡️https://t.co/eUKLgHYGqP https://t.co/bOkLL1FlDp 

TWEET #7 - Tech for Campaigns, created to get Democrats elected, on the parties' biggest differences https://t

## Methods

In [6]:
user = api.get_user("JATayler")

print(f"Screen name is: {user.screen_name}")
print(f"There are {user.followers_count} followers")
print(f"Currently following:")
for friend in user.friends():
   print(f"    - {friend.screen_name}")

Screen name is: JATayler
There are 6372 followers
Currently following:
    - VanHicklestein
    - viewtoakel
    - gfstarr1
    - ms_creilly
    - theshortgirlash
    - manymanywords
    - heyadiana
    - _hkdl_
    - mariokartdwi
    - lalavin666
    - mollylambert
    - normcharlatan
    - jcmccaffrey
    - slwein
    - RichardStaff
    - JasonRRMartinez
    - CurbedNY
    - THECITYNY
    - davelevitan
    - whstancil


## Authentication

There are 2 authentication methods that can be used:
1. OAuth 1a Authentication
    - application-user authenticaiton
2. OAuth 2 Authentication
    - application-only authentication

In [7]:
auth = tw.OAuthHandler(twitter_cred['CONSUMER_KEY'], twitter_cred['CONSUMER_SECRET'])

# For applications that need a callback URL
#auth = tweepy.OAuthHandler(twitter_cred['CONSUMER_KEY'], twitter_cred['CONSUMER_SECRET'], callback_url)


Authentican workflow as follows:

1. Get a request token from twitter
2. Redirect user to twitter.com to authorize application
3. If using callback, twitter will redirect the user to our callback, otherwise, user must suppply verifier code
4. Exchange the authorized request token for an access token

In [8]:
try:
    redirect_url = auth.get_authorization_url()
    print('It works!')
except tweepy.TweepError:
    print('Error! Failed to get request token.')

It works!


## Query Data

In [9]:
feed = {}

for tweet in tw.Cursor(api.search, q='trump', tweet_mode='extended').items(1):
    feed.update(tweet._json)

In [10]:
# What keys are used
parentList = []
subList = []

# Add dictionaries to sub process list
def subProcess(dictionary):
    subList.append(dictionary)
    
for item in feed.keys():
    parentList.append(item)

for i in parentList:
    if type(feed[i]) is not dict:
        print(f"{i} = feed['{i}']")
        
    else:
        subProcess(i)

created_at = feed['created_at']
id = feed['id']
id_str = feed['id_str']
full_text = feed['full_text']
truncated = feed['truncated']
display_text_range = feed['display_text_range']
source = feed['source']
in_reply_to_status_id = feed['in_reply_to_status_id']
in_reply_to_status_id_str = feed['in_reply_to_status_id_str']
in_reply_to_user_id = feed['in_reply_to_user_id']
in_reply_to_user_id_str = feed['in_reply_to_user_id_str']
in_reply_to_screen_name = feed['in_reply_to_screen_name']
geo = feed['geo']
coordinates = feed['coordinates']
place = feed['place']
contributors = feed['contributors']
is_quote_status = feed['is_quote_status']
retweet_count = feed['retweet_count']
favorite_count = feed['favorite_count']
favorited = feed['favorited']
retweeted = feed['retweeted']
possibly_sensitive = feed['possibly_sensitive']
lang = feed['lang']


In [83]:
# Add dictionaries to sub process list
def subProcess(dictionary):
    subList.append(dictionary)

def iterateThrough(sub):
    narrowDown = []
    for i2 in feed[sub].keys():
        narrowDown.append(i2)
        if type(feed['retweeted_status'][sub][i2]) is not dict:
            print(f"retweeted_status_{sub}_{i2} = feed['retweeted_status'['{sub}']['{i2}']")
        else:
            print(i2)      
        
iterateThrough('entities')



retweeted_status_entities_hashtags = feed['retweeted_status'['entities']['hashtags']
retweeted_status_entities_symbols = feed['retweeted_status'['entities']['symbols']
retweeted_status_entities_user_mentions = feed['retweeted_status'['entities']['user_mentions']
retweeted_status_entities_urls = feed['retweeted_status'['entities']['urls']
retweeted_status_entities_media = feed['retweeted_status'['entities']['media']


In [99]:
feed['entities']['media']

[{'id': 1314036895748444160,
  'id_str': '1314036895748444160',
  'indices': [88, 111],
  'media_url': 'http://pbs.twimg.com/media/EjxlmDjX0AA3pOs.jpg',
  'media_url_https': 'https://pbs.twimg.com/media/EjxlmDjX0AA3pOs.jpg',
  'url': 'https://t.co/JZ7EKpYzlQ',
  'display_url': 'pic.twitter.com/JZ7EKpYzlQ',
  'expanded_url': 'https://twitter.com/RJP56/status/1314036898269077506/photo/1',
  'type': 'photo',
  'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
   'medium': {'w': 680, 'h': 398, 'resize': 'fit'},
   'large': {'w': 680, 'h': 398, 'resize': 'fit'},
   'small': {'w': 680, 'h': 398, 'resize': 'fit'}},
  'source_status_id': 1314036898269077506,
  'source_status_id_str': '1314036898269077506',
  'source_user_id': 165472649,
  'source_user_id_str': '165472649'}]

In [85]:
# All of the metadata in a single tweet

created_at = feed['created_at']
twitter_id = feed['id']
id_str = feed['id_str']
full_text = feed['full_text']
truncated = feed['truncated']
display_text_range = feed['display_text_range']
entities = feed['entities']
metadata = feed['metadata']
source = feed['source']
in_reply_to_status_id = feed['in_reply_to_status_id']
in_reply_to_status_id_str = feed['in_reply_to_status_id_str']
in_reply_to_user_id = feed['in_reply_to_user_id']
in_reply_to_user_id_str = feed['in_reply_to_user_id_str']
in_reply_to_screen_name = feed['in_reply_to_screen_name']
user = feed['user']
geo = feed['geo']
coordinates = feed['coordinates']
place = feed['place']
contributors = feed['contributors']
retweeted_status = feed['retweeted_status']
is_quote_status = feed['is_quote_status']
retweet_count = feed['retweet_count']
favorite_count = feed['favorite_count']
favorited = feed['favorited']
retweeted = feed['retweeted']
lang = feed['lang']
user_id = feed['user']['id']
user_id_str = feed['user']['id_str']
user_name = feed['user']['name']
user_screen_name = feed['user']['screen_name']
user_location = feed['user']['location']
user_description = feed['user']['description']
user_url = feed['user']['url']
user_protected = feed['user']['protected']
user_followers_count = feed['user']['followers_count']
user_friends_count = feed['user']['friends_count']
user_listed_count = feed['user']['listed_count']
user_created_at = feed['user']['created_at']
user_favourites_count = feed['user']['favourites_count']
user_utc_offset = feed['user']['utc_offset']
user_time_zone = feed['user']['time_zone']
user_geo_enabled = feed['user']['geo_enabled']
user_verified = feed['user']['verified']
user_statuses_count = feed['user']['statuses_count']
user_lang = feed['user']['lang']
user_contributors_enabled = feed['user']['contributors_enabled']
user_is_translator = feed['user']['is_translator']
user_is_translation_enabled = feed['user']['is_translation_enabled']
user_profile_background_color = feed['user']['profile_background_color']
user_profile_background_image_url = feed['user']['profile_background_image_url']
user_profile_background_image_url_https = feed['user']['profile_background_image_url_https']
user_profile_background_tile = feed['user']['profile_background_tile']
user_profile_image_url = feed['user']['profile_image_url']
user_profile_image_url_https = feed['user']['profile_image_url_https']
user_profile_banner_url = feed['user']['profile_banner_url']
user_profile_link_color = feed['user']['profile_link_color']
user_profile_sidebar_border_color = feed['user']['profile_sidebar_border_color']
user_profile_sidebar_fill_color = feed['user']['profile_sidebar_fill_color']
user_profile_text_color = feed['user']['profile_text_color']
user_profile_use_background_image = feed['user']['profile_use_background_image']
user_has_extended_profile = feed['user']['has_extended_profile']
user_default_profile = feed['user']['default_profile']
user_default_profile_image = feed['user']['default_profile_image']
user_following = feed['user']['following']
user_follow_request_sent = feed['user']['follow_request_sent']
user_notifications = feed['user']['notifications']
user_translator_type = feed['user']['translator_type']
entities_hashtags = feed['entities']['hashtags']
entities_symbols = feed['entities']['symbols']
entities_user_mentions = feed['entities']['user_mentions']
entities_urls = feed['entities']['urls']
entities_media = feed['entities']['media']
extended_entities_media = feed['extended_entities']['media']
metadata_iso_language_code = feed['metadata']['iso_language_code']
metadata_result_type = feed['metadata']['result_type']
retweeted_status_created_at = feed['retweeted_status']['created_at']
retweeted_status_id = feed['retweeted_status']['id']
retweeted_status_id_str = feed['retweeted_status']['id_str']
retweeted_status_full_text = feed['retweeted_status']['full_text']
retweeted_status_truncated = feed['retweeted_status']['truncated']
retweeted_status_display_text_range = feed['retweeted_status']['display_text_range']
retweeted_status_source = feed['retweeted_status']['source']
retweeted_status_in_reply_to_status_id = feed['retweeted_status']['in_reply_to_status_id']
retweeted_status_in_reply_to_status_id_str = feed['retweeted_status']['in_reply_to_status_id_str']
retweeted_status_in_reply_to_user_id = feed['retweeted_status']['in_reply_to_user_id']
retweeted_status_in_reply_to_user_id_str = feed['retweeted_status']['in_reply_to_user_id_str']
retweeted_status_in_reply_to_screen_name = feed['retweeted_status']['in_reply_to_screen_name']
retweeted_status_geo = feed['retweeted_status']['geo']
retweeted_status_coordinates = feed['retweeted_status']['coordinates']
retweeted_status_place = feed['retweeted_status']['place']
retweeted_status_contributors = feed['retweeted_status']['contributors']
retweeted_status_is_quote_status = feed['retweeted_status']['is_quote_status']
retweeted_status_retweet_count = feed['retweeted_status']['retweet_count']
retweeted_status_favorite_count = feed['retweeted_status']['favorite_count']
retweeted_status_favorited = feed['retweeted_status']['favorited']
retweeted_status_retweeted = feed['retweeted_status']['retweeted']
retweeted_status_possibly_sensitive = feed['retweeted_status']['possibly_sensitive']
retweeted_status_lang = feed['retweeted_status']['lang']
retweeted_status_entities_hashtags = feed['retweeted_status']['entities']['hashtags']
retweeted_status_entities_symbols = feed['retweeted_status']['entities']['symbols']
retweeted_status_entities_user_mentions = feed['retweeted_status']['entities']['user_mentions']
retweeted_status_entities_urls = feed['retweeted_status']['entities']['urls']
retweeted_status_entities_media = feed['retweeted_status']['entities']['media']
user_entities_description_urls = feed['user']['entities']['description']['urls']


In [ ]:
# Interesting metadata

# Subject
user['screen_name']
user['name']
user['description']
user['url']
user['followers_count']
user['friends_count']
user['created_at'] # When the account was created
user['verified'] # Social influence
user['statuses_count'] # Number of tweets created since account creation
user['contributors_enabled']
# Chain linking social network / Tracking conversations
entities['user_mentions']['screen_name'] # This could be used to chain link social network
entities['user_mentions'][in_reply_to_screen_name] # This could be used to chain link social network

In [37]:
def acqData(search, acq):

    index_name = search.split(' ')[0] + '-' + dt.today().strftime('%Y-%m-%d')
    feed = []
    
    print('::Acquiring Data::')
   
    for tweet in tw.Cursor(api.search, q=search, tweet_mode='extended').items(acq):
        feed.append(tweet._json)

    count = 0
    
    print('::Transferring to Elasticsearch Search::')
    
    while count < len(feed):
        tweet_date = feed[count]['created_at']
        username = feed[count]['user']['screen_name']
        account_creation_date = feed[count]['created_at']
        user_description = feed[count]['user']['description']
        user_url = feed[count]['user']['url']
        verified_status = feed[count]['user']['verified']
        geo_enabled = feed[count]['user']['geo_enabled']
        friends_count = feed[count]['user']['friends_count']
        followers_count = feed[count]['user']['followers_count']
        retweeted_count = feed[count]['retweet_count']
        favorite_count = feed[count]['favorite_count']
        hashtags = feed[count]['entities']['hashtags']
        tweet_full_text = feed[count]['full_text']

        doc = {
            '@timestamp': dt.now(),
            'tweet_date': tweet_date,
            'username': str(username),
            'account_creation_date': str(account_creation_date),
            'user_description': str(user_description),
            'user_url': str(user_url),
            'verified_status': bool(verified_status),
            'geo_enabled': bool(geo_enabled),
            'friends_count': int(friends_count),
            'followers_count': int(followers_count),
            'retweeted_count': int(retweeted_count),
            'favorite_count': int(favorite_count),
            'hashtags': hashtags,
            'tweet_full_text': str(tweet_full_text),
            'word_list': str(tweet_full_text).split(' ')
        }

        es.index(index=index_name, body=doc)
        print(f'{count}: {doc}')
        
        count +=1

In [38]:
acqData('palantir OR PLTR', 100)

::Acquiring Data::
::Transferring to Elasticsearch Search::
0: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 43, 270709), 'tweet_date': 'Tue Oct 06 07:06:37 +0000 2020', 'username': 'lindadavies56', 'account_creation_date': 'Tue Oct 06 07:06:37 +0000 2020', 'user_description': '', 'user_url': 'None', 'verified_status': False, 'geo_enabled': False, 'friends_count': 582, 'followers_count': 419, 'retweeted_count': 22, 'favorite_count': 0, 'hashtags': [], 'tweet_full_text': 'RT @StefSimanowitz: 6/. On 8 July, the UK govt sidelined SAGE in favour of the secretive Joint Biosecurity Centre headed by a senior spy\n\nO…', 'word_list': ['RT', '@StefSimanowitz:', '6/.', 'On', '8', 'July,', 'the', 'UK', 'govt', 'sidelined', 'SAGE', 'in', 'favour', 'of', 'the', 'secretive', 'Joint', 'Biosecurity', 'Centre', 'headed', 'by', 'a', 'senior', 'spy\n\nO…']}
1: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 43, 288355), 'tweet_date': 'Tue Oct 06 07:04:49 +0000 2020', 'username': 'StockAlertsA

14: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 43, 468150), 'tweet_date': 'Tue Oct 06 06:44:14 +0000 2020', 'username': 'KoichiTsunoda', 'account_creation_date': 'Tue Oct 06 06:44:14 +0000 2020', 'user_description': 'ヤプリってノーコードSaaSの会社で取締役CFOとベンチプレスやってます。採用したい → iOS/Android/Goエンジニア、広報 | @UCBerkeley → 外資金融 → CFO @マナボ（売却）| Living a startup dream in Tokyo. No-code B2B SaaS.', 'user_url': 'None', 'verified_status': False, 'geo_enabled': False, 'friends_count': 985, 'followers_count': 8164, 'retweeted_count': 0, 'favorite_count': 6, 'hashtags': [], 'tweet_full_text': '直接上場は広まってほしいけど、全てを解決する魔法じゃないしもっと色んな選択肢があって良い。SPACも話題だけど主幹事をつけながら自社でアロケーションしたUnityのIPOも気になる\n\nなぜAsanaやPalantir、Spotify、Slackは伝統的IPOを回避したのか？ https://t.co/CgAnRYHLr1 via @coral_capital', 'word_list': ['直接上場は広まってほしいけど、全てを解決する魔法じゃないしもっと色んな選択肢があって良い。SPACも話題だけど主幹事をつけながら自社でアロケーションしたUnityのIPOも気になる\n\nなぜAsanaやPalantir、Spotify、Slackは伝統的IPOを回避したのか？', 'https://t.co/CgAnRYHLr1', 'via', '@coral_capital']}
15: {'@timestamp': datetim

34: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 43, 683426), 'tweet_date': 'Tue Oct 06 06:18:42 +0000 2020', 'username': 'NanaB_2010', 'account_creation_date': 'Tue Oct 06 06:18:42 +0000 2020', 'user_description': 'Psych major, Behavior Analyst  #BlackLivesMatter #Autism  #PTSD #BP #SpecialOlympics #Advocacy #Climate #Equality', 'user_url': 'None', 'verified_status': False, 'geo_enabled': False, 'friends_count': 1530, 'followers_count': 829, 'retweeted_count': 6, 'favorite_count': 0, 'hashtags': [], 'tweet_full_text': 'RT @MijenteComite: For the past two weeks, folks have been chasing them across the country in protests demanding they cut the ICE contracts…', 'word_list': ['RT', '@MijenteComite:', 'For', 'the', 'past', 'two', 'weeks,', 'folks', 'have', 'been', 'chasing', 'them', 'across', 'the', 'country', 'in', 'protests', 'demanding', 'they', 'cut', 'the', 'ICE', 'contracts…']}
35: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 43, 695155), 'tweet_date': 'Tue Oct 06 06

53: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 43, 890571), 'tweet_date': 'Tue Oct 06 06:03:01 +0000 2020', 'username': 'pitbullet8', 'account_creation_date': 'Tue Oct 06 06:03:01 +0000 2020', 'user_description': '元T企業勤務 ＞ FIRE中。脳内リセットしてランダムに学びながら、米国ETF・個別株の中長期投資。投資は学び続けるための一種のモチベーション', 'user_url': 'None', 'verified_status': False, 'geo_enabled': False, 'friends_count': 103, 'followers_count': 44, 'retweeted_count': 0, 'favorite_count': 0, 'hashtags': [], 'tweet_full_text': 'https://t.co/0laBeyss9A', 'word_list': ['https://t.co/0laBeyss9A']}
54: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 43, 901549), 'tweet_date': 'Tue Oct 06 05:59:22 +0000 2020', 'username': 'itarumusic', 'account_creation_date': 'Tue Oct 06 05:59:22 +0000 2020', 'user_description': '@kai_ogita とジャーナリストや専門家、作家などの個人ブランド向けに、月額ニュースレター配信サービス『theLetter (https://t.co/n7ZEF5HbKC)』を運営してます。タトゥー、音楽、映画、小説、二郎が好き。ジャーナリズムと個人のマネタイズについて日々研究しています。エッセイは下記リンクより', 'user_url': 'https://t.co/5pYM9XxEUW', 'verified_statu

73: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 44, 99199), 'tweet_date': 'Tue Oct 06 05:53:51 +0000 2020', 'username': 'blackrepublican', 'account_creation_date': 'Tue Oct 06 05:53:51 +0000 2020', 'user_description': 'Authentic #blackconservatism has always had the fundamental and explicit goal of opposing white supremacy.\n\n— Kareim Oliphant\n\n#BlackConservative', 'user_url': 'https://t.co/8b7EAlLWPg', 'verified_status': False, 'geo_enabled': False, 'friends_count': 88070, 'followers_count': 71891, 'retweeted_count': 14, 'favorite_count': 0, 'hashtags': [], 'tweet_full_text': "RT @carolineha_: Here's an excerpt, showing the scale of Palantir use within the LAPD, and some of the colleges + school districts that sha…", 'word_list': ['RT', '@carolineha_:', "Here's", 'an', 'excerpt,', 'showing', 'the', 'scale', 'of', 'Palantir', 'use', 'within', 'the', 'LAPD,', 'and', 'some', 'of', 'the', 'colleges', '+', 'school', 'districts', 'that', 'sha…']}
74: {'@timestamp': datetime.date

92: {'@timestamp': datetime.datetime(2020, 10, 6, 14, 6, 44, 307246), 'tweet_date': 'Tue Oct 06 05:34:27 +0000 2020', 'username': 'blackrepublican', 'account_creation_date': 'Tue Oct 06 05:34:27 +0000 2020', 'user_description': 'Authentic #blackconservatism has always had the fundamental and explicit goal of opposing white supremacy.\n\n— Kareim Oliphant\n\n#BlackConservative', 'user_url': 'https://t.co/8b7EAlLWPg', 'verified_status': False, 'geo_enabled': False, 'friends_count': 88070, 'followers_count': 71891, 'retweeted_count': 194, 'favorite_count': 0, 'hashtags': [], 'tweet_full_text': 'RT @StefSimanowitz: On 8 July, the UK govt sidelined SAGE in favour of the secretive Joint Biosecurity Centre headed by a senior spy.\n\nOn 2…', 'word_list': ['RT', '@StefSimanowitz:', 'On', '8', 'July,', 'the', 'UK', 'govt', 'sidelined', 'SAGE', 'in', 'favour', 'of', 'the', 'secretive', 'Joint', 'Biosecurity', 'Centre', 'headed', 'by', 'a', 'senior', 'spy.\n\nOn', '2…']}
93: {'@timestamp': datetim

In [ ]:
tweets = []
for tweet in tweepy.Cursor(api.search, q='#PLTR').items(10):
    tweets.append(tweet._json) 


In [ ]:
import pandas as pd
df = pd.read_csv('test.csv')

In [ ]:
tweets[0]['user']['name']

In [ ]:
tweets[0]['text']

In [ ]:
created = tweets[0]['user']['created_at']
screen_name = tweets[0]['user']['screen_name']
name = tweets[0]['user']['name']
followers_count = tweets[0]['user']['followers_count']
friends_count = tweets[0]['user']['friends_count']
geo = tweets[0]['geo']
coordinates = tweets[0]['coordinates']
place = tweets[0]['place']
retweet_count = tweets[0]['retweet_count']
tweet_text = tweets[0]['text']

friends_list = []
followers_list = []

user = api.get_user(screen_name)
for friend in user.friends():
   friends_list.append(friend.screen_name)

In [ ]:
len(friends_list)

In [ ]:
print(f'{created}\n \
        Screen Name: {screen_name}\n \
        Real Name: {name}\n \
        Number of Followers {followers_count}\n \
        Number of Friends {friends_count}\n \
        Number of Retweets: {retweet_count}\n \
        Location: {geo}, {coordinates}, {place}\n \
        Friends List: {friends_list}\n \
        Tweet: {tweet_text}\n')

In [ ]:
import datetime as dt
search = "palantir OR PLTR"
date_begin = (dt.datetime.today() - dt.timedelta(days=7)).strftime("%Y-""%m-""%d")

In [ ]:
tweets = tw.Cursor(api.search, q = search, tweet_mode = "extended", lang = "en", since = date_begin).items(2)

In [ ]:
feed = []
for tweet in tweets:
    feed.append(tweet)

In [ ]:
import json
json_data = json.loads(feed[0])